# Proyecto Final


- Alfredo Osuna
- Raul Gonzalez

## Librerias Necesarias

In [10]:
(require '[clojure.string :as s])
(require '[clojupyter.misc.helper :as helper])
(helper/add-dependencies '[net.mikera/core.matrix "0.62.0"])
(helper/add-dependencies '[net.mikera/vectorz-clj "0.48.0"])
(set-current-implementation :vectorz)
(use 'clojure.core.matrix)
(defn toInt[x] (Integer/parseInt x));Java
(defn toDouble[x] (Double/parseDouble x));Java
(helper/add-dependencies '[incanter "1.5.7"])

{[incanter/incanter-io "1.5.7"] #{[net.sf.opencsv/opencsv "2.3"] [org.danlarkin/clojure-json "1.1" :exclusions [[org.clojure/clojure] [org.clojure/clojure-contrib]]]}, [org.scilab.forge/jlatexmath "0.9.6"] nil, [net.sf.opencsv/opencsv "2.3"] nil, [org.apache.xmlbeans/xmlbeans "2.3.0"] #{[stax/stax-api "1.0.1"]}, [net.sourceforge.f2j/arpack_combined_all "0.1"] nil, [xalan "2.6.0"] nil, [org.clojure/data.json "0.2.0"] nil, [com.lowagie/itext "2.1.7"] #{[bouncycastle/bcprov-jdk14 "138"] [org.bouncycastle/bctsp-jdk14 "1.38"] [bouncycastle/bcmail-jdk14 "138"]}, [net.sourceforge.parallelcolt/parallelcolt "0.10.1"] #{[com.github.rwl/JKLU "1.0.0"] [net.sourceforge.csparsej/csparsej "1.1.1"] [com.googlecode.netlib-java/netlib-java "0.9.3"] [net.sourceforge.parallelcolt/optimization "1.0"] [net.sourceforge.jtransforms/jtransforms "2.4.0"] [net.sourceforge.jplasma/jplasma "1.2.0"]}, [com.github.rwl/JKLU "1.0.0"] #{[com.github.rwl/BTFJ "1.0.1"] [com.github.rwl/AMDJ "1.0.1"] [com.github.rwl/COLAMDJ

## Clase de Pelicula

In [11]:
(defrecord Movie [x0 imdb runtime rottenTomato]
    Object 
    (toString [this]
              (format "%s,%s,%s,%s" x0 imdb runtime rottenTomato)
              )
    )

user.Movie

## Lectura de informacion

In [12]:
(def data (vec (rest (s/split-lines (slurp "Clean-Data.csv") ) ) ) )

#'user/data

## Conversion de datos

In [13]:
(defn toMovie[ record ]
    (Movie. 
         (toInt(get record 0))  
         (toDouble (get record 1))
         (toInt (get record 2)) 
         (toDouble (get record 3))
     )
)

#'user/toMovie

In [14]:
(defn get-Movies-lazy [datos] 
    (lazy-seq (cons (toMovie (s/split (first datos) #","))(get-Movies-lazy (rest datos))) )
    )

#'user/get-Movies-lazy

In [15]:
(defn get-Vector-Valor [datos, valor]
    (lazy-seq (cons (valor (first datos))(get-Vector-Valor (rest datos) valor)))
    )

#'user/get-Vector-Valor

In [16]:
(def movies (take 5120 (get-Movies-lazy data)))

#'user/movies

In [17]:
(def imdb (vec (take 5120 (get-Vector-Valor movies :imdb))))

#'user/imdb

In [18]:
(def rottenTomato (vec (take 5120 (get-Vector-Valor movies :rottenTomato))))

#'user/rottenTomato

In [19]:
(def runtime (vec (take 5120 (get-Vector-Valor movies :runtime))))

#'user/runtime

In [20]:
(def x0 (vec (take 5120 (get-Vector-Valor movies :x0))))

#'user/x0

In [21]:
(def  matrix_transpuesta [x0 imdb runtime])

#'user/matrix_transpuesta

## Transpuesta Xt

In [22]:
(def matrixA (transpose matrix_transpuesta))

#'user/matrixA

## Multiplicador de matrices (Xt X)


In [23]:
(def matrix_multiplicacion_xt_x (mmul matrix_transpuesta matrixA))

#'user/matrix_multiplicacion_xt_x

In [24]:
matrix_multiplicacion_xt_x

[[5120.0 32641.600000000042 512570.0] [32641.600000000042 213247.28 3290089.9000000055] [512570.0 3290089.9000000055 5.3714518E7]]

## Inversa (Xt X)^-1

In [25]:
(def matrix_inversa (inverse matrix_multiplicacion_xt_x))

#'user/matrix_inversa

In [26]:
matrix_inversa

[[0.010373355596556475 -0.0011023659305894101 -3.146612734737078E-5] [-0.0011023659305894101 2.0243777987766342E-4 -1.8802884894491774E-6] [-3.146612734737078E-5 -1.8802884894491774E-6 4.340523182701685E-7]]

## Multiplicacion Xt Y

In [27]:
(def matrix_multiplicacion_xt_y (mmul matrix_transpuesta rottenTomato))

#'user/matrix_multiplicacion_xt_y

In [28]:
 matrix_multiplicacion_xt_y

[3348.1400000000067 22189.450999999986 335297.04000000126]

## Multiplicacion (Xt X)^-1 (Xt Y)

In [29]:
(def pesos (mmul matrix_inversa matrix_multiplicacion_xt_y))

#'user/pesos

In [30]:
pesos

[-0.2799473536649515 0.17065256544217766 -0.001539111398196763]

In [31]:
(def x1 10)
(def x2 106)

#'user/x2

In [32]:
(+ (get pesos 0)(* (get pesos 1) x1)(* (get pesos 2) x2))

1.2634324925479683

## Grafica

In [43]:
(require '[clojupyter.misc.helper :as helper])
(helper/add-dependencies '[hswick/jutsu "0.1.1"])
(require '[jutsu.core :as j])
(j/start-jutsu! )

Starting http-kit...


"http://localhost:1040/"

In [37]:
(def imdb_g (apply list imdb))

#'user/imdb_g

In [38]:
(def runtime_g (apply list runtime))

#'user/runtime_g

In [40]:
(def rottenTomato_g (apply list rottenTomato))

#'user/rottenTomato_g

In [41]:
(def a (get pesos 0))
(def b (get pesos 1))
(def c (get pesos 2))

#'user/c

In [42]:
(def y_pred (map + (take (count imdb_g) (repeat a)) (map #(* % b) imdb_g) (map #(* % c) rottenTomatto_g) ) )

#'user/y_pred

In [44]:
(def scatter-graph3d [{:x imdb_g
                       :y runtime_g
                       :z y_pred
                       :mode "markers"
                       :type "scatter3d"}])
(j/graph! "Price data" scatter-graph3d)

1